In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum, round

# 1. Завантажте та прочитайте кожен CSV-файл як окремий DataFrame.

In [69]:
spark = SparkSession.builder.appName("ProductAnalysis").getOrCreate()
purchases = spark.read.csv("purchases.csv", header=True, inferSchema=True)
products = spark.read.csv("products.csv", header=True, inferSchema=True)
users = spark.read.csv("users.csv", header=True, inferSchema=True)

In [71]:
purchases.show(5)

+-----------+-------+----------+----------+--------+
|purchase_id|user_id|product_id|      date|quantity|
+-----------+-------+----------+----------+--------+
|          1|     52|         9|2022-01-01|       1|
|          2|     93|        37|2022-01-02|       8|
|          3|     15|        33|2022-01-03|       1|
|          4|     72|        42|2022-01-04|       9|
|          5|     61|        44|2022-01-05|       6|
+-----------+-------+----------+----------+--------+
only showing top 5 rows



In [74]:
products.show(5), users.show(5)

+----------+------------+-----------+-----+
|product_id|product_name|   category|price|
+----------+------------+-----------+-----+
|         1|   Product_1|     Beauty|  8.3|
|         2|   Product_2|       Home|  8.3|
|         3|   Product_3|Electronics|  9.2|
|         4|   Product_4|Electronics|  2.6|
|         5|   Product_5|Electronics|  9.4|
+----------+------------+-----------+-----+
only showing top 5 rows

+-------+------+---+-----------------+
|user_id|  name|age|            email|
+-------+------+---+-----------------+
|      1|User_1| 45|user1@example.com|
|      2|User_2| 48|user2@example.com|
|      3|User_3| 36|user3@example.com|
|      4|User_4| 46|user4@example.com|
|      5|User_5| 29|user5@example.com|
+-------+------+---+-----------------+
only showing top 5 rows



(None, None)

# 2. Очистіть дані, видаляючи будь-які рядки з пропущеними значеннями.

In [ ]:
purchases_clean = purchases.dropna()
products_clean = products.dropna()
users_clean = users.dropna()

In [72]:
purchases_clean.show(5)

+-----------+-------+----------+----------+--------+
|purchase_id|user_id|product_id|      date|quantity|
+-----------+-------+----------+----------+--------+
|          1|     52|         9|2022-01-01|       1|
|          2|     93|        37|2022-01-02|       8|
|          3|     15|        33|2022-01-03|       1|
|          4|     72|        42|2022-01-04|       9|
|          5|     61|        44|2022-01-05|       6|
+-----------+-------+----------+----------+--------+
only showing top 5 rows



In [73]:
products_clean.show(5), users_clean.show(5)

+----------+------------+-----------+-----+
|product_id|product_name|   category|price|
+----------+------------+-----------+-----+
|         1|   Product_1|     Beauty|  8.3|
|         2|   Product_2|       Home|  8.3|
|         3|   Product_3|Electronics|  9.2|
|         4|   Product_4|Electronics|  2.6|
|         5|   Product_5|Electronics|  9.4|
+----------+------------+-----------+-----+
only showing top 5 rows

+-------+------+---+-----------------+
|user_id|  name|age|            email|
+-------+------+---+-----------------+
|      1|User_1| 45|user1@example.com|
|      2|User_2| 48|user2@example.com|
|      3|User_3| 36|user3@example.com|
|      4|User_4| 46|user4@example.com|
|      5|User_5| 29|user5@example.com|
+-------+------+---+-----------------+
only showing top 5 rows



(None, None)

# 3. Визначте загальну суму покупок за кожною категорією продуктів.

In [ ]:
merged_data = purchases_clean.join(products_clean, on="product_id")
merged_data.show(5)

+----------+-----------+-------+----------+--------+------------+-----------+-----+
|product_id|purchase_id|user_id|      date|quantity|product_name|   category|price|
+----------+-----------+-------+----------+--------+------------+-----------+-----+
|         9|          1|     52|2022-01-01|       1|   Product_9|     Beauty|  6.0|
|        37|          2|     93|2022-01-02|       8|  Product_37|   Clothing|  6.0|
|        33|          3|     15|2022-01-03|       1|  Product_33|       Home|  9.4|
|        42|          4|     72|2022-01-04|       9|  Product_42|     Beauty|  9.1|
|        24|          6|     21|2022-01-06|       7|  Product_24|Electronics|  2.1|
+----------+-----------+-------+----------+--------+------------+-----------+-----+
only showing top 5 rows



In [ ]:
merged_data = merged_data.withColumn("purchase_amount", round(col("quantity") * col("price"), 2))
merged_data.show(5)

+----------+-----------+-------+----------+--------+------------+-----------+-----+---------------+
|product_id|purchase_id|user_id|      date|quantity|product_name|   category|price|purchase_amount|
+----------+-----------+-------+----------+--------+------------+-----------+-----+---------------+
|         9|          1|     52|2022-01-01|       1|   Product_9|     Beauty|  6.0|            6.0|
|        37|          2|     93|2022-01-02|       8|  Product_37|   Clothing|  6.0|           48.0|
|        33|          3|     15|2022-01-03|       1|  Product_33|       Home|  9.4|            9.4|
|        42|          4|     72|2022-01-04|       9|  Product_42|     Beauty|  9.1|           81.9|
|        24|          6|     21|2022-01-06|       7|  Product_24|Electronics|  2.1|           14.7|
+----------+-----------+-------+----------+--------+------------+-----------+-----+---------------+
only showing top 5 rows



In [ ]:
total_by_category = merged_data.groupBy("category").agg(
    round(spark_sum("purchase_amount"), 2).alias("total_purchase")
)
total_by_category.show()

+-----------+--------------+
|   category|total_purchase|
+-----------+--------------+
|       Home|        1523.5|
|     Sports|        1802.5|
|Electronics|        1174.8|
|   Clothing|         790.3|
|     Beauty|         459.9|
+-----------+--------------+



# 4. Визначте суму покупок за кожною категорією продуктів для вікової категорії від 18 до 25 включно.

In [ ]:
merged_with_users = merged_data.join(users_clean, on="user_id")
merged_with_users.show(5)

+-------+----------+-----------+----------+--------+------------+-----------+-----+---------------+-------+---+------------------+
|user_id|product_id|purchase_id|      date|quantity|product_name|   category|price|purchase_amount|   name|age|             email|
+-------+----------+-----------+----------+--------+------------+-----------+-----+---------------+-------+---+------------------+
|     52|         9|          1|2022-01-01|       1|   Product_9|     Beauty|  6.0|            6.0|User_52| 39|user52@example.com|
|     93|        37|          2|2022-01-02|       8|  Product_37|   Clothing|  6.0|           48.0|User_93| 25|user93@example.com|
|     15|        33|          3|2022-01-03|       1|  Product_33|       Home|  9.4|            9.4|User_15| 30|user15@example.com|
|     72|        42|          4|2022-01-04|       9|  Product_42|     Beauty|  9.1|           81.9|User_72| 39|user72@example.com|
|     21|        24|          6|2022-01-06|       7|  Product_24|Electronics|  2.1|

In [ ]:
age_filtered = merged_with_users.filter((col("age") >= 18) & (col("age") <= 25))
age_filtered.show(5)

+-------+----------+-----------+----------+--------+------------+--------+-----+---------------+-------+---+------------------+
|user_id|product_id|purchase_id|      date|quantity|product_name|category|price|purchase_amount|   name|age|             email|
+-------+----------+-----------+----------+--------+------------+--------+-----+---------------+-------+---+------------------+
|     93|        37|          2|2022-01-02|       8|  Product_37|Clothing|  6.0|           48.0|User_93| 25|user93@example.com|
|     88|        41|         11|2022-01-11|       8|  Product_41|  Sports|  5.9|           47.2|User_88| 25|user88@example.com|
|     88|        49|         18|2022-01-18|       3|  Product_49|  Sports|  9.7|           29.1|User_88| 25|user88@example.com|
|     58|        37|         27|2022-01-27|       6|  Product_37|Clothing|  6.0|           36.0|User_58| 20|user58@example.com|
|     91|        19|         31|2022-01-31|       6|  Product_19|Clothing|  3.5|           21.0|User_91|

In [ ]:
total_by_category_age_group = age_filtered.groupBy("category").agg(
    round(spark_sum("purchase_amount"), 2).alias("total_purchase")
)
total_by_category_age_group.show(5)

+-----------+--------------+
|   category|total_purchase|
+-----------+--------------+
|       Home|         361.1|
|     Sports|         310.5|
|Electronics|         249.6|
|   Clothing|         245.0|
|     Beauty|          41.4|
+-----------+--------------+



# 5. Визначте частку покупок за кожною категорією товарів від сумарних витрат для вікової категорії від 18 до 25 років.

In [ ]:
total_purchase_age_group = age_filtered.agg(
    spark_sum("purchase_amount").alias("total_purchase")
).collect()[0]["total_purchase"]
print(total_purchase_age_group)

1207.6


In [ ]:
category_percentage = total_by_category_age_group.withColumn(
    "percentage", round((col("total_purchase") / total_purchase_age_group) * 100, 2)
)

category_percentage.show()

+-----------+--------------+----------+
|   category|total_purchase|percentage|
+-----------+--------------+----------+
|       Home|         361.1|      29.9|
|     Sports|         310.5|     25.71|
|Electronics|         249.6|     20.67|
|   Clothing|         245.0|     20.29|
|     Beauty|          41.4|      3.43|
+-----------+--------------+----------+



# 6. Виберіть 3 категорії продуктів з найвищим відсотком витрат споживачами віком від 18 до 25 років.

In [ ]:
top_3_categories = category_percentage.orderBy(col("percentage").desc()).limit(3)

top_3_categories.show()

+-----------+--------------+----------+
|   category|total_purchase|percentage|
+-----------+--------------+----------+
|       Home|         361.1|      29.9|
|     Sports|         310.5|     25.71|
|Electronics|         249.6|     20.67|
+-----------+--------------+----------+

